# Docetaxel Sensitivity: Machine Learning using Gene Expression
---------------------------

In [3]:
import numpy as np
import pandas as pd
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

In [4]:
## Get Sensitivity Data
doce_data = Dataset.get_by_name(ws, name='docetaxel_sensitivity')
doce_data_df = doce_data.to_pandas_dataframe()
doce_data_df

,L10_IC_50,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,ARHGAP11B,AC004593.2,AC090517.4,AL160269.1,ABCF2-H2BE1,POLR2J3,H2BE1,AL445238.1,GET1-SH3BGR,AC113348.1
0,-1.6643720768426125,2.643856189774725,0.0,6.219555769166913,3.418189947945766,4.659924558402379,0.01435529297707,0.1110313123887439,5.846242565109514,7.057883448899626,...,2.111031312388744,0.0565835283663675,0.0426443374084937,0.0565835283663675,1.1243281350022014,5.069959890459774,0.0,0.0,1.803227036434928,0.0
1,-2.2657961042166934,2.985500430304885,0.0,6.778734244117836,4.130930869826448,3.778208576398088,0.0,0.2986583155645151,7.43379405875492,6.689299160535892,...,1.7355221772965377,0.3448284969974411,0.3673710656485294,0.1243281350022017,2.192194165283345,4.358958825832329,0.0,0.0426443374084937,0.0976107966264223,0.0
2,-2.194771295116583,4.574707046415546,0.0,6.632413641137546,1.9373443921502325,3.4019034716079584,0.0285691521967709,0.5753123306874368,5.7755773609306535,3.320484678017693,...,2.4776773275653072,1.2203299548795556,0.1110313123887439,0.0,2.84197311892718,3.6158870739160567,0.0,0.0,1.1890338243900171,0.0
3,-2.8168508023444905,5.868637384170314,0.0,6.636044526028037,2.0461417816447205,4.9963887464476215,0.1763227726404629,1.655351828612554,6.200457271428572,3.4982508675278257,...,0.7739963251111731,0.0285691521967709,0.137503523749935,0.070389327891398,1.1763227726404628,6.022811650915631,0.0,0.0,0.0565835283663675,0.0
4,-2.370915780677939,5.026800059343715,0.0,6.96613048992893,1.8991756304805127,3.5310694927259543,0.0,3.910732661902913,6.371384613557957,4.693208148910016,...,1.9523335663696857,0.0426443374084937,0.0426443374084937,0.0426443374084937,3.104336659814736,5.9342805936400245,0.0,0.0,0.3785116232537298,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,-2.4501423858336175,0.0,0.0,6.23112515787905,2.6848187375532224,4.898208352508718,3.0268000593437154,0.0285691521967709,3.3476656563009706,3.7729413378313352,...,2.533563348214512,0.0,0.0565835283663675,0.0,1.9068905956085185,5.387500406480984,0.0,0.0565835283663675,0.4005379295837288,0.0
677,-0.24498841253817064,5.649328040984784,0.0,6.6748278321868,3.085764553778314,3.790772037862,0.2265085298086797,3.878725341480105,6.167518503735217,5.101818134227409,...,2.0840642647884744,0.2016338611696504,0.0,0.0,2.3757345385831563,4.361768359419154,0.0,0.0,0.3334237337251918,0.0
678,-2.6648063309377252,4.863938450423972,0.0,5.977966757619478,2.553360503335328,3.7729413378313352,0.2750070474998698,7.17881421635862,6.867278739709662,3.959770155211468,...,3.3103401206121505,0.0,0.0,0.0,3.748461233004035,6.263973354803409,0.0,0.0,0.1110313123887439,0.0
679,-3.0343703784319263,5.9140860970127225,0.0,6.749668340903956,2.8094144442358986,4.175524601089875,0.1763227726404629,5.859224161978204,6.535275376620803,4.598126959919604,...,2.263034405833794,0.1634987322828795,0.0976107966264223,0.4956951626240688,3.3798981635246923,6.094869432718697,0.0,0.0,1.0285691521967708,0.0


## Create Training Script

In [5]:
%%writefile train.py
## Import libraries
import argparse
import joblib
import os
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

## Set the input parameters
parser = argparse.ArgumentParser()
parser.add_argument("--n_estimators", type=int, dest='n_estimators', help='Number of gradient boosted trees.')
parser.add_argument("--max_depth", type=int, dest='max_depth', help='Maximum tree depth for base learners.')
parser.add_argument("--booster", type=str, dest='booster', help='Boosting method.')
args = parser.parse_args()

n_estimators = args.n_estimators
max_depth = args.max_depth
booster = args.booster

## Get the experiment run context
run = Run.get_context()
ws = run.experiment.workspace

## Log run options
run.log('n_estimators', str(n_estimators))
run.log('max_depth', str(max_depth))
run.log('booster', str(booster))

## load the dataset(s)
print("Loading Data...")
dataset = run.input_datasets['doce_data'].to_pandas_dataframe() # Get the training data from the estimator input
run.log("Input data shape: ", str(dataset.shape))


## Separate features and labels
X = dataset[dataset.columns[1:]]
y = dataset[['L10_IC_50']].values

## Get X as numpy array
X = X.values

## Scale the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)


## Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.25, 
    random_state = 1337
)

## Initialize Algorithm
xgb = XGBRegressor(objective ='reg:linear',
                   n_estimators = n_estimators,
                   max_depth = max_depth,
                   booster = booster,
                   seed = 1337)

## Train Model w/ Cross-Validation (5-fold)
scores = cross_validate(xgb,
                        X_train, y_train,
                        cv=5, scoring=('r2'),
                        return_estimator=True)

model = scores['estimator'][0]

## Score data
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

## Log model performance
run.log('r2', r2_score(y_train, y_train_hat))
run.log('r2_test', r2_score(y_test, y_test_hat))

## Note: Files saved in the outputs folder is automatically uploaded into experiment record
os.makedirs('outputs', exist_ok=True)

## Save model
model_name = f'model_{filtering_method}_{scaling_method}_{algorithm}'
joblib.dump(value=model, filename = f'outputs/{model_name}.pkl')

run.complete()

Overwriting train.py


## Compute Target

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-cluster1"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=4)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster.


## Configure Hyperdrive

In [7]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, RandomParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice, uniform


# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed (we need scikit-learn, Azure ML defaults, and Azure ML dataprep)
packages = CondaDependencies.create(pip_packages=['scikit-learn','azureml-defaults','azureml-dataprep[pandas]', 'xgboost'])
sklearn_env.python.conda_dependencies = packages

## Create a script config
script_config = ScriptRunConfig(source_directory = ".",
                                script='train.py',
                                arguments = ['--doce_data', doce_data.as_named_input('doce_data')], # Reference to dataset
                                environment = sklearn_env,
                                compute_target = training_cluster)

## Sample a range of parameter values
# params = GridParameterSampling(
params = RandomParameterSampling(
    {
        '--n_estimators': choice(1, 5, 100),
        '--max_depth': uniform(100, 5000),
        '--booster': choice('gbtree', 'gblinear', 'dart')
    }
)

## Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config = script_config, 
                              hyperparameter_sampling = params, 
                              policy = None, 
                              primary_metric_name = 'r2', 
                              primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                              max_total_runs = 10,
                              max_concurrent_runs = 4)

## Run the Experiment

In [8]:
experiment = Experiment(workspace = ws, name = 'docetaxel_sensitivity_training_hyperdrive')
run = experiment.submit(config = hyperdrive)

In [ ]:
from azureml.widgets import RunDetails

## Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…